## 인스타그램 수집

In [2]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.keys import Keys 
import time 
import pandas as pd 
from datetime import datetime 
from bs4 import BeautifulSoup 

# 계정 정보
INSTAGRAM_ID = "ddukbbangii_i"
INSTAGRAM_PW = "qawsed123" 

# Chrome 드라이버 설정
options = Options() 
options.add_argument("--start-maximized") 
driver = webdriver.Chrome(options=options) 

# 로그인
driver.get("https://www.instagram.com/") 
time.sleep(5)

try: 
    id_input = driver.find_element(By.XPATH, '//*[@id="loginForm"]/div[1]/div[1]/div/label/input') 
    id_input.send_keys(INSTAGRAM_ID) 
    time.sleep(1) 
    pw_input = driver.find_element(By.XPATH, '//*[@id="loginForm"]/div[1]/div[2]/div/label/input') 
    pw_input.send_keys(INSTAGRAM_PW) 
    time.sleep(1) 
    login_btn = driver.find_element(By.XPATH, '//*[@id="loginForm"]/div[1]/div[3]/button') 
    login_btn.click() 
    print("🔐 로그인 시도 중...") 
    time.sleep(7) 
except Exception as e: 
    print(f"❌ 로그인 실패: {e}") 

# 채널 페이지로 이동 
target_url = "https://www.instagram.com/playoverwatch/" 
driver.get(target_url) 
time.sleep(5) 

# 게시물 URL 수집
def collect_post_urls(max_posts=200): 
    """스크롤하면서 게시물 URL들을 수집""" 
    print(f"📜 최대 {max_posts}개 게시물 URL 수집 중...") 
    collected_urls = set() # 중복 방지 
    last_height = driver.execute_script("return document.body.scrollHeight") 
    no_new_posts_count = 0 
    
    while len(collected_urls) < max_posts and no_new_posts_count < 3: 
        # 현재 페이지에 있는 모든 게시물 링크 찾기 
        try: 
            # 여러 선택자로 시도 
            selectors = [ 
                "a.x1i10hfl[href*='/p/']", 
                "a[href*='/p/']", 
            ] 
            current_posts = [] 
            for selector in selectors: 
                try: 
                    posts = driver.find_elements(By.CSS_SELECTOR, selector) 
                    if posts: 
                        current_posts = posts 
                        break 
                except: 
                    continue 

            # URL 수집 
            before_count = len(collected_urls) 
            for post in current_posts: 
                try: 
                    href = post.get_attribute("href") 
                    if href and ("/p/" in href or "/reel/" in href): 
                        collected_urls.add(href) 
                except: 
                    continue 

            after_count = len(collected_urls) 
            new_found = after_count - before_count 
            print(f"📊 현재까지 수집된 URL: {len(collected_urls)}개 (새로 발견: {new_found}개)") 

            if new_found == 0: 
                no_new_posts_count += 1 
                print(f"⚠️ 새로운 게시물 없음 ({no_new_posts_count}/3)") 
            else: 
                no_new_posts_count = 0 

            # 스크롤 
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
            time.sleep(4) 

            # 페이지 높이 확인 
            new_height = driver.execute_script("return document.body.scrollHeight") 
            if new_height == last_height: 
                no_new_posts_count += 1 
                print(f"⚠️ 페이지 높이 변화 없음 ({no_new_posts_count}/3)") 
            else: 
                last_height = new_height 
                
        except Exception as e: 
            print(f"❌ URL 수집 중 오류: {e}") 
            break 

    print(f"✅ URL 수집 완료: 총 {len(collected_urls)}개") 
    return list(collected_urls) 

# 게시물 URL 수집
post_urls = collect_post_urls(max_posts=100) 
if not post_urls: 
    print("❌ 게시물 URL을 찾을 수 없습니다.") 
    driver.quit() 
    exit() 

# 각 게시물 방문하여 데이터 수집
results = [] 
stop_date = datetime.strptime("2024.07.01", "%Y.%m.%d") 
print(f"\n🔄 {len(post_urls)}개 게시물 데이터 수집 시작...") 

for i, post_url in enumerate(post_urls, 1): 
    try: 
        print(f"📸 [{i}/{len(post_urls)}] 게시물 처리 중: {post_url}") 
        
        # 게시물 페이지로 직접 이동 
        driver.get(post_url) 
        time.sleep(5) 

        # 본문 수집 (실제 게시물 페이지 구조에 맞게 수정)
        content = ""
        try:
            print(f"🔍 본문 수집 시도 중...")
            
            # 실제 게시물 페이지의 구조에 맞는 선택자들
            content_selectors = [
                # 실제 구조
                "div.html-div.xdj266r.x14z9mp.xat24cr.x1lziwak.xexx8yu.xyri2b.x18d9i69.x1c1uobl.x9f619.xjbqb8w.x78zum5.x15mokao.x1ga7v0g.x16uus16.xbiv7yw.x1uhb9sk.x1plvlek.xryxfnj.x1iyjqo2.x2lwn1j.xeuugli.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1 span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp.xo1l8bm.x5n08af.x10wh9bi.xpm28yp.x8viiok.x1o7cslx.x126k92a",
                
                # 간단한 버전들
                "div.html-div span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.xt0psk2",
                "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp.xo1l8bm.x5n08af.x10wh9bi.xpm28yp.x8viiok.x1o7cslx.x126k92a",
                "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.xt0psk2",
                "span.xt0psk2.x193iq5w",
                
                # 주요 클래스만으로
                "span.xt0psk2",
                "span.x193iq5w",
                "span.xeuugli.xt0psk2",
                
                # div.html-div 내부 모든 span
                "div.html-div span",
                "div.xdj266r span",
                
                # 더 광범위한 검색
                "div[class*='html-div'] span[class*='xt0psk2']",
                "div[class*='xdj266r'] span[class*='x193iq5w']",
            ]
            
            for selector in content_selectors:
                try:
                    print(f"  CSS 시도: {selector}")
                    elements = driver.find_elements(By.CSS_SELECTOR, selector)
                    print(f"    발견된 요소 수: {len(elements)}개")
                    
                    for i, element in enumerate(elements):
                        text = element.text.strip()
                        print(f"    [{i+1}] 텍스트: {text[:80]}...")
                        
                        # 본문 조건 체크
                        if (text and 
                            len(text) > 10 and 
                            text != "..." and
                            "좋아요" not in text and 
                            "like" not in text.lower() and
                            "팔로우" not in text and 
                            "follow" not in text.lower() and
                            "댓글" not in text and 
                            "comment" not in text.lower() and
                            "공유" not in text and 
                            "저장" not in text and
                            "더 보기" not in text and
                            not text.replace(",", "").replace(".", "").replace("만", "").isdigit()):
                            
                            content = text
                            print(f"✅ 본문 발견! 선택자: {selector}")
                            print(f"   길이: {len(text)}자, 내용: {text[:100]}...")
                            break
                    
                    if content:
                        break
                        
                except Exception as e:
                    print(f"    CSS 실패: {e}")
                    continue
            
            # CSS로 못 찾으면 XPath로 시도
            if not content:
                print("🔍 XPath로 재시도...")
                xpath_selectors = [
                    # 클래스 기반 XPath
                    "//div[contains(@class, 'html-div')]//span[contains(@class, 'xt0psk2')]",
                    "//div[contains(@class, 'xdj266r')]//span[contains(@class, 'x193iq5w')]",
                    "//span[contains(@class, 'xt0psk2') and contains(@class, 'x193iq5w')]",
                    "//span[contains(@class, 'xt0psk2')]",
                    "//span[contains(@class, 'x193iq5w')]",
                    
                    # 긴 텍스트가 있는 span
                    "//span[string-length(text()) > 15]",
                    "//div[contains(@class, 'html-div')]//span[string-length(text()) > 15]",
                ]
                
                for xpath in xpath_selectors:
                    try:
                        print(f"  XPath 시도: {xpath}")
                        elements = driver.find_elements(By.XPATH, xpath)
                        print(f"    발견된 요소 수: {len(elements)}개")
                        
                        for i, element in enumerate(elements):
                            text = element.text.strip()
                            print(f"    [{i+1}] 텍스트: {text[:80]}...")
                            
                            if (text and 
                                len(text) > 15 and 
                                text != "..." and
                                "좋아요" not in text and 
                                "팔로우" not in text and
                                "댓글" not in text and
                                not text.replace(",", "").isdigit()):
                                
                                content = text
                                print(f"✅ XPath로 본문 발견!")
                                break
                        
                        if content:
                            break
                            
                    except Exception as e:
                        print(f"    XPath 실패: {e}")
                        continue
            
            # 마지막: 모든 span 요소 확인
            if not content:
                print("🔍 모든 span 요소 확인...")
                try:
                    all_spans = driver.find_elements(By.TAG_NAME, "span")
                    print(f"    총 {len(all_spans)}개의 span 요소 발견")
                    
                    for i, span in enumerate(all_spans):
                        try:
                            text = span.text.strip()
                            classes = span.get_attribute("class") or ""
                            
                            # 본문으로 보이는 긴 텍스트
                            if (text and 
                                len(text) > 20 and 
                                len(text) < 1000 and
                                text != "..." and
                                "좋아요" not in text and 
                                "팔로우" not in text and
                                "댓글" not in text and
                                ("xt0psk2" in classes or "x193iq5w" in classes or 
                                 "Step" in text or "Destiny" in text or "Nine" in text)):
                                
                                content = text
                                print(f"✅ span 탐색으로 본문 발견! 인덱스: {i+1}")
                                print(f"   클래스: {classes}")
                                print(f"   텍스트: {text[:100]}...")
                                break
                                
                        except Exception as e:
                            continue
                            
                except Exception as e:
                    print(f"  모든 span 확인 실패: {e}")
                    
        except Exception as e:
            print(f"❌ 본문 수집 전체 실패: {e}")
            content = ""
                    
        except Exception as e:
            print(f"⚠️ 본문 수집 실패: {e}")
            content = ""

        # 좋아요 수 수집 
        likes = "" 
        try: 
            # 여러 방법으로 좋아요 수 찾기 
            like_selectors = [ 
                "a[href*='liked_by']", 
                "span:contains('좋아요')", 
                "button span:contains('좋아요')", 
            ] 
            for selector in like_selectors: 
                try: 
                    if "contains" in selector: 
                        # XPath 사용 
                        xpath = f"//span[contains(text(), '좋아요')]" 
                        like_elem = driver.find_element(By.XPATH, xpath) 
                    else: 
                        like_elem = driver.find_element(By.CSS_SELECTOR, selector) 
                    likes = like_elem.text 
                    break 
                except: 
                    continue 

            # 좋아요 수를 찾지 못했을 때 숫자가 있는 span 찾기 
            if not likes: 
                try: 
                    spans = driver.find_elements(By.CSS_SELECTOR, "span") 
                    for span in spans: 
                        span_text = span.text.strip() 
                        if span_text and any(char.isdigit() for char in span_text): 
                            if "좋아요" in span_text or "like" in span_text.lower() or span_text.replace(',', '').replace('.', '').replace('만', '').isdigit(): 
                                likes = span_text 
                                break 
                except: 
                    pass 
        except: 
            likes = "" 

        # 게시일 수집 
        date_obj = None 
        try: 
            time_tag = WebDriverWait(driver, 5).until( 
                EC.presence_of_element_located((By.CSS_SELECTOR, "time")) 
            ) 
            date_text = time_tag.get_attribute("title") 
            if date_text: 
                # 한국어 날짜 형식 처리 
                date_text = date_text.replace("년 ", "-").replace("월 ", "-").replace("일", "") 
                date_obj = datetime.strptime(date_text, "%Y-%m-%d") 
                
                # 기준일 이전이면 종료 
                if date_obj < stop_date: 
                    print(f"🛑 기준일({stop_date.date()}) 이전 게시물 발견 → 종료") 
                    break 
        except Exception as e: 
            print(f"⚠️ 날짜 파싱 실패: {e}") 
            date_obj = None 

        print(f"✅ [{i}] 게시일: {date_obj.date() if date_obj else ''} | 좋아요: {likes} | 본문: {content[:30]}...") 
        
        results.append({ 
            "게시일": date_obj.date() if date_obj else "", 
            "좋아요 수": likes, 
            "본문": content, 
            "주소": post_url 
        }) 

        # 10개마다 진행 상황 출력 
        if i % 10 == 0: 
            print(f"🔄 {i}개 수집 완료") 
            
    except Exception as e: 
        print(f"⚠️ 게시물 {i} 처리 실패: {e}") 
        continue 

# 9. 결과 저장
try:
    insta_df = pd.DataFrame(results)
    insta_df.to_csv("13-1. OW2_instagram_crawled.csv", index=False, encoding="utf-8-sig")
    print(f"\n✅ 저장 완료: 13-1. OW2_instagram_crawled.csv (총 {len(insta_df)}개)")
except Exception as e:
    print(f"❌ 저장 실패: {e}")
finally:
    driver.quit()

🔐 로그인 시도 중...
📜 최대 100개 게시물 URL 수집 중...
📊 현재까지 수집된 URL: 3개 (새로 발견: 3개)
📊 현재까지 수집된 URL: 4개 (새로 발견: 1개)
📊 현재까지 수집된 URL: 7개 (새로 발견: 3개)
📊 현재까지 수집된 URL: 11개 (새로 발견: 4개)
📊 현재까지 수집된 URL: 13개 (새로 발견: 2개)
📊 현재까지 수집된 URL: 13개 (새로 발견: 0개)
⚠️ 새로운 게시물 없음 (1/3)
📊 현재까지 수집된 URL: 14개 (새로 발견: 1개)
📊 현재까지 수집된 URL: 18개 (새로 발견: 4개)
📊 현재까지 수집된 URL: 21개 (새로 발견: 3개)
📊 현재까지 수집된 URL: 26개 (새로 발견: 5개)
📊 현재까지 수집된 URL: 31개 (새로 발견: 5개)
📊 현재까지 수집된 URL: 37개 (새로 발견: 6개)
📊 현재까지 수집된 URL: 37개 (새로 발견: 0개)
⚠️ 새로운 게시물 없음 (1/3)
📊 현재까지 수집된 URL: 41개 (새로 발견: 4개)
📊 현재까지 수집된 URL: 45개 (새로 발견: 4개)
📊 현재까지 수집된 URL: 50개 (새로 발견: 5개)
📊 현재까지 수집된 URL: 52개 (새로 발견: 2개)
📊 현재까지 수집된 URL: 54개 (새로 발견: 2개)
📊 현재까지 수집된 URL: 58개 (새로 발견: 4개)
📊 현재까지 수집된 URL: 60개 (새로 발견: 2개)
📊 현재까지 수집된 URL: 65개 (새로 발견: 5개)
📊 현재까지 수집된 URL: 71개 (새로 발견: 6개)
📊 현재까지 수집된 URL: 78개 (새로 발견: 7개)
📊 현재까지 수집된 URL: 81개 (새로 발견: 3개)
📊 현재까지 수집된 URL: 82개 (새로 발견: 1개)
📊 현재까지 수집된 URL: 83개 (새로 발견: 1개)
📊 현재까지 수집된 URL: 85개 (새로 발견: 2개)
📊 현재까지 수집된 URL: 92개 (새로 발견: 7개)
📊 현재까지 수집된 URL: 92개 (새로 발견:

## 유튜브 수집
* 수집항목 : 제목, 주소(추후 댓글 크롤링 위함), 조회수, 게시일(n개월 전)

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time

options = Options()
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)

# 링크
driver.get("https://www.youtube.com/@playoverwatch/videos")
time.sleep(3)

# 영상 수집
video_data = []
scroll_pause = 2
last_height = driver.execute_script("return document.documentElement.scrollHeight")
stop_crawling = False

while not stop_crawling:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    video_items = soup.select("ytd-rich-item-renderer")

    for item in video_items[len(video_data):]:  # 중복 방지
        title_tag = item.select_one("#video-title")
        link_tag = item.select_one("a#video-title-link")
        meta_tags = item.select("span.inline-metadata-item")

        if title_tag and link_tag and len(meta_tags) > 1:
            title = title_tag.text.strip()
            url = "https://www.youtube.com" + link_tag["href"]
            posted = meta_tags[1].text.strip()
            views = meta_tags[0].text.strip()

            # 종료 조건: 게시일이 '1년 전'
            if "1년 전" in posted:
                print(f"🛑 게시일 '{posted}' → 크롤링 종료")
                stop_crawling = True
                break

            print(f"📹 {title} | {url} | {views} | {posted}")
            video_data.append({
                "제목": title,
                "URL": url,
                "조회수": views,
                "게시일": posted
            })

    # 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(scroll_pause)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
driver.quit()

# 6. CSV로 저장
youtube_df = pd.DataFrame(video_data)
csv_path = "13-2. OW2_youtube_crawled.csv"
youtube_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"\n✅ 총 {len(youtube_df)}개 영상 저장 완료: {csv_path}")

📹 Season 18 Mythic Gameplay Trailer | Overwatch 2 | https://www.youtube.com/watch?v=x2nomKpZBGo | 조회수 8.4만회 | 3주 전
📹 Developer Update | Season 18: Wuyang, Progression 2.0, Stadium Updates, & More | https://www.youtube.com/watch?v=UrCSmaCZHY0 | 조회수 17만회 | 4주 전
📹 Elemental Kin | Wuyang Hero Trailer | Overwatch 2 | https://www.youtube.com/watch?v=ytZnOGwEq9s | 조회수 26만회 | 1개월 전
📹 Season 18: Stadium Quick Play Official Trailer | Overwatch 2 | https://www.youtube.com/watch?v=r2nwpbXoDdY | 조회수 32만회 | 1개월 전
📹 Wuyang | New Hero Gameplay Trailer | Overwatch 2 | https://www.youtube.com/watch?v=C6ZEN5MVJaU | 조회수 69만회 | 1개월 전
📹 Wuyang Story Time with Howard Wang | Overwatch 2 | https://www.youtube.com/watch?v=RY3sER2O0jM | 조회수 30만회 | 1개월 전
📹 Overwatch 2 x NERF | Gameplay Trailer | https://www.youtube.com/watch?v=X_q9JoVcImI | 조회수 34만회 | 1개월 전
📹 Overwatch 2 x G.I. Joe | Gameplay Trailer | https://www.youtube.com/watch?v=jGIDpGzEiRw | 조회수 26만회 | 2개월 전
📹 Developer Update | Season 17: Map Updates, Stad

In [5]:
youtube_df = pd.read_csv("13-2. OW2_youtube_crawled.csv", encoding="utf-8-sig")
yturl_list = youtube_df['URL'].tolist()

In [6]:
yturl_list

['https://www.youtube.com/watch?v=x2nomKpZBGo',
 'https://www.youtube.com/watch?v=UrCSmaCZHY0',
 'https://www.youtube.com/watch?v=ytZnOGwEq9s',
 'https://www.youtube.com/watch?v=r2nwpbXoDdY',
 'https://www.youtube.com/watch?v=C6ZEN5MVJaU',
 'https://www.youtube.com/watch?v=RY3sER2O0jM',
 'https://www.youtube.com/watch?v=X_q9JoVcImI',
 'https://www.youtube.com/watch?v=jGIDpGzEiRw',
 'https://www.youtube.com/watch?v=NKJi5oKQTf8',
 'https://www.youtube.com/watch?v=7p5K_d1qgfQ&pp=0gcJCeAJAYcqIYzv',
 'https://www.youtube.com/watch?v=9kFSMbaufn8',
 'https://www.youtube.com/watch?v=aKR-7k-AZQ4',
 'https://www.youtube.com/watch?v=schtk917iRU',
 'https://www.youtube.com/watch?v=dZoYO4hUYUk',
 'https://www.youtube.com/watch?v=Rwnt3O4cEew',
 'https://www.youtube.com/watch?v=l0rnlqYFmiM&pp=0gcJCeAJAYcqIYzv',
 'https://www.youtube.com/watch?v=OugMrXvdv6k',
 'https://www.youtube.com/watch?v=E9tUpO0BTQY',
 'https://www.youtube.com/watch?v=GM85xnXH6JQ',
 'https://www.youtube.com/watch?v=LriEejDFSnQ',


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re, time
import pandas as pd

options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# 숫자 변환 함수
def to_int(text):
    if not text:
        return None
    text = text.upper()
    multiplier = 1
    if 'K' in text:
        multiplier = 1000; text = text.replace('K', '')
    elif 'M' in text:
        multiplier = 1000000; text = text.replace('M', '')
    elif '만' in text:
        multiplier = 10000; text = text.replace('만', '')
    elif '억' in text:
        multiplier = 100000000; text = text.replace('억', '')
    m = re.search(r'([\d,\.]+)', text.replace('\u00a0', ' '))
    if not m: return None
    try:
        return int(float(m.group(1).replace(',', '')) * multiplier)
    except:
        return None

# 댓글 수 수집
def get_comment_count(driver):
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.3);")
        time.sleep(2)
        WebDriverWait(driver, 8).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ytd-comments-header-renderer"))
        )
        selectors = [
            "ytd-comments-header-renderer h2#count yt-formatted-string span[dir='auto']",
            "ytd-comments-header-renderer h2#count yt-formatted-string span",
            "ytd-comments-header-renderer #count yt-formatted-string",
            "ytd-comments-header-renderer h2#count span",
            "ytd-comments-header-renderer .count-text span",
            "ytd-comments-header-renderer .count-text",
        ]
        for selector in selectors:
            try:
                text = driver.find_element(By.CSS_SELECTOR, selector).text.strip()
                count = to_int(text)
                if count is not None: 
                    return count
            except: 
                continue
    except:
        pass
    return 0

# 크롤링
like_counts, comment_counts, clean_urls = [], [], []

for i, url in enumerate(yturl_list, 1):  # yturl_list는 미리 정의되어 있어야 함
    like_val, cmt_val = None, None
    try:
        driver.get(url)
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "ytd-app"))
        )
        time.sleep(2)

        # 좋아요
        try:
            btns = driver.find_elements(By.XPATH, "//div[@id='top-level-buttons-computed']//button[@aria-label]")
            for b in btns:
                label = (b.get_attribute("aria-label") or "").lower()
                if any(k in label for k in ["좋아", "like"]):
                    like_val = to_int(label)
                    if like_val is not None:
                        break
        except: pass
        if like_val is None:
            try:
                el = driver.find_element(By.XPATH,
                    "//ytd-segmented-like-dislike-button-renderer//yt-smartimation//span[@id='text' or @id='button']"
                )
                like_val = to_int(el.text)
            except: pass

        # 댓글
        cmt_val = get_comment_count(driver)

    except:
        like_val, cmt_val = None, 0

    clean_urls.append(url)
    like_counts.append(like_val)
    comment_counts.append(cmt_val)

    # 결과만 출력
    print(f"[{i}/{len(yturl_list)}] {url}, 좋아요 수: {like_val}, 댓글 수: {cmt_val}")

driver.quit()

# --- DataFrame 저장 ---
yt_meta = pd.DataFrame({
    "URL": clean_urls,
    "좋아요 수": like_counts,
    "댓글 수": comment_counts
})

base_df = pd.read_csv("13-2. OW2_youtube_crawled.csv", encoding="utf-8-sig")
enriched = base_df.merge(yt_meta, on="URL", how="left")
enriched.to_csv("13-3. OW2_youtube_crawled_complete.csv", index=False, encoding="utf-8-sig")
print("저장 완료: 13-3. OW2_youtube_crawled_complete.csv")

[1/44] https://www.youtube.com/watch?v=x2nomKpZBGo, 좋아요 수: 3452, 댓글 수: 399
[2/44] https://www.youtube.com/watch?v=UrCSmaCZHY0, 좋아요 수: 13146, 댓글 수: 2138
[3/44] https://www.youtube.com/watch?v=ytZnOGwEq9s, 좋아요 수: 19715, 댓글 수: 1653
[4/44] https://www.youtube.com/watch?v=r2nwpbXoDdY, 좋아요 수: 13945, 댓글 수: 1614
[5/44] https://www.youtube.com/watch?v=C6ZEN5MVJaU, 좋아요 수: 30395, 댓글 수: 3385
[6/44] https://www.youtube.com/watch?v=RY3sER2O0jM, 좋아요 수: 17301, 댓글 수: 2026
[7/44] https://www.youtube.com/watch?v=X_q9JoVcImI, 좋아요 수: 15925, 댓글 수: 1470
[8/44] https://www.youtube.com/watch?v=jGIDpGzEiRw, 좋아요 수: 9884, 댓글 수: 1615
[9/44] https://www.youtube.com/watch?v=NKJi5oKQTf8, 좋아요 수: 8843, 댓글 수: 1005
[10/44] https://www.youtube.com/watch?v=7p5K_d1qgfQ&pp=0gcJCeAJAYcqIYzv, 좋아요 수: 18930, 댓글 수: 3217
[11/44] https://www.youtube.com/watch?v=9kFSMbaufn8, 좋아요 수: 3460, 댓글 수: 425
[12/44] https://www.youtube.com/watch?v=aKR-7k-AZQ4, 좋아요 수: 22461, 댓글 수: 2461
[13/44] https://www.youtube.com/watch?v=schtk917iRU, 좋아요 수:

### 업데이트기록

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

# ▶ URL 설정
url = "https://store.steampowered.com/news/app/2357570"

# ▶ 드라이버 설정
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# ▶ 페이지 열기
driver.get(url)
time.sleep(3)

# ▶ 수집용 set
collected_urls = set()
results = []

# ▶ 천천히 스크롤하며 수집
scroll_count = 0
max_scrolls = 50
no_new_count = 0

while scroll_count < max_scrolls and no_new_count < 3:
    soup = BeautifulSoup(driver.page_source, "html.parser")

    news_blocks = soup.find_all("div", class_="_398u23KF15gxmeH741ZSyL")

    found_this_round = 0

    for block in news_blocks:
        a_tag = block.find("a", class_="Focusable")
        if a_tag:
            href = a_tag.get("href")
            if href and href not in collected_urls:
                full_url = "https://store.steampowered.com" + href
                collected_urls.add(href)
                results.append({"링크": full_url})
                print(f"📝 [{len(results)}] {full_url}")
                found_this_round += 1

    if found_this_round == 0:
        no_new_count += 1
    else:
        no_new_count = 0

    # 스크롤 다운
    driver.execute_script("window.scrollBy(0, 1500);")
    scroll_count += 1
    time.sleep(2.5)

# ▶ 저장
updatelink_df = pd.DataFrame(results)
updatelink_df['링크'] = updatelink_df["링크"].str.extract(r"(\/view\/\d+)")
updatelink_df.to_csv("13-1. steam_update_links.csv", index=False, encoding="utf-8-sig")
print(f"\n✅ 총 {len(updatelink_df)}개 뉴스 링크 저장 완료 → 13-1. steam_update_links.csv")

driver.quit()


📝 [1] https://store.steampowered.com/news/app/2357570/view/609788303381633012
📝 [2] https://store.steampowered.com/news/app/2357570/view/519714314320347168
📝 [3] https://store.steampowered.com/news/app/2357570/view/519714314320347166
📝 [4] https://store.steampowered.com/news/app/2357570/view/644688664507450470
📝 [5] https://store.steampowered.com/news/app/2357570/view/519712411641446603
📝 [6] https://store.steampowered.com/news/app/2357570/view/519711143023870668
📝 [7] https://store.steampowered.com/news/app/2357570/view/519711143023870377
📝 [8] https://store.steampowered.com/news/app/2357570/view/519711143023870109
📝 [9] https://store.steampowered.com/news/app/2357570/view/519711143023869953
📝 [10] https://store.steampowered.com/news/app/2357570/view/836088461785040100
📝 [11] https://store.steampowered.com/news/app/2357570/view/534344040319027055
📝 [12] https://store.steampowered.com/news/app/2357570/view/497187441695785007
📝 [13] https://store.steampowered.com/news/app/2357570/view/5

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# ▶ 드라이버 세팅
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# ▶ URL 베이스
base_url = "https://store.steampowered.com/news/app/2357570"

# ▶ update_df["링크"]에 '/view/숫자' 형태가 들어있다고 가정
# 예시용 (실제에는 update_df = pd.read_csv(...)로 불러와야 함)
# update_df = pd.read_csv("steam_news_links_stable.csv")
# update_df["링크"] = update_df["링크"].str.extract(r"(\/view\/\d+)")

results = []

for i, update_link in enumerate(updatelink_df["링크"], 1):
    full_url = base_url + update_link
    print(f"\n🌐 [{i}] 크롤링 중: {full_url}")

    try:
        driver.get(full_url)
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # ✅ 제목
        title_tag = soup.select_one("div._3z2NYCkFizMu4fMvWTIBUG div.TqEPC9bhvVpZ1rb3Z8Mbd")
        title = title_tag.text.strip() if title_tag else ""

        # ✅ 게시일
        date_tag = soup.select_one("div._3IxVZE9uydjh3cA9kmtnk7 div._2KsEbGy9kiSDeQpcqEc9DG div._1Maw_Rw6sOKYC1KkOI1xM")
        date = date_tag.text.strip() if date_tag else ""

        # ✅ 유형
        type_tag = soup.select_one("div._3phfIcOe_STA7hSoFfIxlE")
        type_text = type_tag.text.strip() if type_tag else ""

        # ✅ 본문
        body_tag = soup.select_one("div.EventDetailsBody.A_A2B6fTn_MPLlGCmsLtd")
        body = body_tag.text.strip() if body_tag else ""

        # ✅ 좋아요 수
        like_tag = soup.select_one("div._9x4Z7eMgdwfAVMr16ZaJ0 div._3Kelh1-_v6xHfRjF68n7NB div._3csl-MPe-hKuT8hQpOqEG5")
        likes = like_tag.text.strip() if like_tag else ""

        # ✅ 토론 주소
        comment_tag = soup.select_one("div._16xC0mtOWoLbvSQbmo_ycv a.Focusable")
        comment_link = comment_tag["href"] if comment_tag else ""

        # ▶ 저장
        results.append({
            "링크": full_url,
            "제목": title,
            "게시일": date,
            "유형": type_text,
            "본문": body,
            "좋아요": likes,
            "댓글 주소": comment_link
        })

        print(f"📝 제목: {title} | 게시일: {date} | 좋아요: {likes} | 본문: {body[:20]}")

    except Exception as e:
        print(f"❌ [{i}] 오류 발생: {e}")
        continue

driver.quit()

# ▶ DataFrame 저장
update_df = pd.DataFrame(results)
update_df.to_csv("13-2. steam_update_details.csv", index=False, encoding="utf-8-sig")
print(f"\n✅ 총 {len(df)}개 뉴스 상세 정보 저장 완료 → 13-2. steam_update_details.csv")



🌐 [1] 크롤링 중: https://store.steampowered.com/news/app/2357570/view/609788303381633012
📝 제목: Overwatch 2 Season 17: Powered Up! is Now Live | 게시일: StartedThu, June 26, 2025 2:35 AM KST | 좋아요: 1,836 | 본문: Prepare for a power 

🌐 [2] 크롤링 중: https://store.steampowered.com/news/app/2357570/view/519714314320347168
📝 제목: S16 - Midseason Update | 게시일: Posted Wed, May 21, 2025 @12:40 PM KST | 좋아요: 632 | 본문: Season 16 midseason 

🌐 [3] 크롤링 중: https://store.steampowered.com/news/app/2357570/view/519714314320347166
📝 제목: Overwatch 2 Retail Patch Notes – May 20, 2025 | 게시일: Posted Wed, May 21, 2025 @12:31 PM KST | 좋아요: 314 | 본문: Stadium Continues!Mi

🌐 [4] 크롤링 중: https://store.steampowered.com/news/app/2357570/view/644688664507450470
📝 제목: Overwatch 2 x Street Fighter 6 | 게시일: Posted Tue, May 20, 2025 @1:00 AM KST | 좋아요: 1,051 | 본문: Two iconic worlds co

🌐 [5] 크롤링 중: https://store.steampowered.com/news/app/2357570/view/519712411641446603
📝 제목: Director’s Take - Past, Present, and Future of Stadium 

In [ ]:
import pandas as pd
update_df = pd.read_csv("13-2. steam_update_details.csv", encoding="utf-8-sig")
reviewlink = update_df['댓글 주소'].str.extract(r"(/eventcomments/.*)").iloc[:, 0].tolist()

In [42]:
reviewlink

['/eventcomments/597404578618583913?snr=1_2108_9__2107',
 '/eventcomments/597401407894595051?snr=1_2108_9__2107',
 '/eventcomments/597401407894594863?snr=1_2108_9__2107',
 '/eventcomments/598527098280278166?snr=1_2108_9__2107',
 '/eventcomments/597399610072295591?snr=1_2108_9__2107',
 '/eventcomments/597398761189217334?snr=1_2108_9__2107',
 '/eventcomments/597398453436341029?snr=1_2108_9__2107',
 '/eventcomments/597398236592935677?snr=1_2108_9__2107',
 '/eventcomments/597398236592865636?snr=1_2108_9__2107',
 '/eventcomments/509577917527788151?snr=1_2108_9__2107',
 '/eventcomments/595143167557307810?snr=1_2108_9__2107',
 '/eventcomments/597392533415305336?snr=1_2108_9__2107',
 '/eventcomments/599643064665939597?snr=1_2108_9__2107',
 '/eventcomments/4425435723661512229?snr=1_2108_9__2107',
 '/eventcomments/4513254014346211763?snr=1_2108_9__2107',
 '/eventcomments/4517757463775333331?snr=1_2108_9__2107',
 '/eventcomments/6895657216184214893?snr=1_2108_9__2107',
 '/eventcomments/4415298372

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# 1. Chrome 설정
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# 2. 댓글 주소 리스트
reviewlink

# 3. 결과 저장 리스트
nickname_all = []
comment_all = []
link_origin = []
base_link_all = []  # 기본 링크 정보
page_number_all = []  # 페이지 번호 정보

# 4. 각 댓글 링크 반복
for single_link in reviewlink:
    url = f"https://steamcommunity.com/app/2357570{single_link}"
    driver.get(url)
    time.sleep(3)
    print(f"\n🚀 댓글 링크 진입: {single_link}")

    current_page = 1  # 페이지 1부터 시작

    while True:
        print(f"   🔄 페이지 {current_page} 수집 중...")
        time.sleep(2)

        # 댓글 섹션 파싱
        comment_boxes = driver.find_elements(By.CLASS_NAME, "commentthread_comment")
        
        page_comment_count = 0  # 현재 페이지에서 수집한 댓글 수

        for box in comment_boxes:
            try:
                nickname = box.find_element(By.CSS_SELECTOR, "a.hoverunderline.commentthread_author_link").text.strip()
                comment = box.find_element(By.CSS_SELECTOR, "div.commentthread_comment_text").text.strip()

                # 현재 페이지의 전체 URL 저장 (페이지 정보 포함)
                current_full_url = driver.current_url
                
                nickname_all.append(nickname)
                comment_all.append(comment)
                link_origin.append(current_full_url)  # 실제 현재 페이지 URL 저장
                base_link_all.append(single_link)  # 기본 링크 정보
                page_comment_count += 1

                print(f"   📝 {nickname}: {comment[:30]}...")

            except Exception as e:
                print(f"   ⚠️ 댓글 파싱 실패: {e}")
                continue

        print(f"   ✅ 페이지 {current_page}에서 {page_comment_count}개 댓글 수집 완료")

        # 다음 페이지로 이동 시도
        try:
            # 다음 페이지 번호 계산
            next_page = current_page + 1
            
            # 다음 페이지 버튼 XPath 정의 (페이지 번호에 따라 다름)
            if next_page <= 6:
                # 페이지 2~6은 직접 번호로 클릭
                xpath = f'//*[@id="commentthread_ForumTopic_103582791473914114_3808407060880330147_597401407894595051_fpagelinks"]/a[text()="{next_page}"]'
            else:
                # 6 이후는 ">" 버튼 클릭
                xpath = '//*[@id="commentthread_ForumTopic_103582791473914114_3808407060880330147_597401407894595051_fpagelinks"]/a[contains(text(), ">")]'

            next_btn = driver.find_element(By.XPATH, xpath)
            
            # 버튼이 클릭 가능한지 확인
            if next_btn.is_enabled():
                driver.execute_script("arguments[0].scrollIntoView(true);", next_btn)
                time.sleep(1)
                next_btn.click()
                current_page = next_page
                time.sleep(2)  # 페이지 로딩 대기
            else:
                print(f"   ✅ 마지막 페이지 도달. 다음 댓글 링크로 이동")
                break

        except Exception as e:
            print(f"   ✅ 더 이상 페이지가 없음. 다음 댓글 링크로 이동 (오류: {e})")
            break

# 5. 저장
if nickname_all:  # 데이터가 있을 때만 저장
    reviewlink_df = pd.DataFrame({
        "닉네임": nickname_all,
        "댓글 본문": comment_all,
        "전체 URL": link_origin,  # 페이지 정보가 포함된 전체 URL
        "기본 링크": base_link_all,  # 원본 링크 정보
    })

    reviewlink_df.to_csv("13-3. steam_update_comments.csv", index=False, encoding="utf-8-sig")
    print(f"\n📁 13-3. steam_update_comments.csv 저장 완료 (총 {len(nickname_all)}개 댓글)")
else:
    print("\n⚠️ 수집된 댓글이 없습니다.")

# 6. 종료
driver.quit()


🚀 댓글 링크 진입: /eventcomments/597404578618583913?snr=1_2108_9__2107
   🔄 페이지 1 수집 중...
   📝 Zombie115: where is artwork tab huh ?...
   📝 Julia: Delete Sombra...
   📝 Gan: 25 gig update, for essentially...
   📝 bill receba la ele da silva: Marvel Rivals>OW2...
   📝 comrade_dario: less annoying marvel fan...
   📝 mixo: Yo bring artwork tab i wanna g...
   📝 Fenzuik: wow same kind of skins League ...
   📝 Libre: Dont touch sombra, pls...
   📝 :P: trash ahh game...
   📝 AvrorA: Sort things out with belarusia...
   📝 melancholia: L battle pass...
   📝 Greenclown: ...
   📝 Maxtraxv3: boring....
   📝 Acpm123: Orisa mythic weapon skin next ...
   📝 Ποιος είμαι εγώ;: Howd you break ForceKeyboard u...
   ✅ 페이지 1에서 15개 댓글 수집 완료
   ✅ 더 이상 페이지가 없음. 다음 댓글 링크로 이동 (오류: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="commentthread_ForumTopic_103582791473914114_3808407060880330147_597401407894595051_fpagelinks"]/a[text()="2"]"}
  (Session info: chrome=138.0.7204

In [48]:
update_df['댓글 주소'] = update_df['댓글 주소'].str.extract(r"(/eventcomments/.*)").iloc[:, 0]

In [ ]:
update_df.to_csv("13-2. steam_update_details.csv", index=False, encoding="utf-8-sig")

In [52]:
reviewlink_df.drop(columns=['전체 URL'], inplace=True)

In [ ]:
reviewlink_df.to_csv('13-3. steam_update_comments.csv', index=False, encoding='utf-8-sig')

In [56]:
reviewlink_df

,닉네임,댓글 본문,기본 링크
0,Zombie115,where is artwork tab huh ?,/eventcomments/597404578618583913?snr=1_2108_9...
1,Julia,Delete Sombra,/eventcomments/597404578618583913?snr=1_2108_9...
2,Gan,"25 gig update, for essentially 2 chars and one...",/eventcomments/597404578618583913?snr=1_2108_9...
3,bill receba la ele da silva,Marvel Rivals>OW2,/eventcomments/597404578618583913?snr=1_2108_9...
4,comrade_dario,less annoying marvel fan,/eventcomments/597404578618583913?snr=1_2108_9...
...,...,...,...
1270,Yumia Liessfeldt,finally,/eventcomments/3808407060880330150?snr=1_2108_...
1271,Unhappygnomeviking,Interesting Addition to steam - it's nice to s...,/eventcomments/3808407060880330150?snr=1_2108_...
1272,One-HiT KiLLeR,"Nice, always wanted to try it.",/eventcomments/3808407060880330150?snr=1_2108_...
1273,sour,be fr,/eventcomments/3808407060880330150?snr=1_2108_...
